# 多練習其他參數版本，觀察其效率變化
## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)->(Batch Size , Height , Width , Channels) 

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3)) #計算(全局)平均值
        std = np.std(X_train, axis=(0, 1, 2, 3)) #計算(全局)標準差
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) #1e-7用於避免除以0
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
#資料預處理
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

D:\Users\u9593\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [9]:
'''自己決定MaxPooling2D放在哪裡'''
#maxpool 採一般CNN放法
classifier=Sequential()
input_shape=(32,32,3)
#卷積組合
classifier.add(Convolution2D(filters=32,kernel_size=(3,3),padding='same',input_shape=input_shape))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization(axis=1,momentum=0.99,epsilon=0.001,beta_initializer='zeros', gamma_initializer='ones'))

classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=64,kernel_size=(3,3),padding='same'))
classifier.add(BatchNormalization(axis=1,momentum=0.99,epsilon=0.001,beta_initializer='zeros', gamma_initializer='ones'))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100,activation='relu')) #output_dim=100,activation=relu

#輸出->過Softmax轉換成機率值
classifier.add(Dense(units=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy(多分类损失函数)
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 31s 614us/step - loss: 1.6747 - acc: 0.4101
Epoch 2/100
50000/50000 [==============================] - 9s 184us/step - loss: 1.2723 - acc: 0.5492
Epoch 3/100
50000/50000 [==============================] - 9s 184us/step - loss: 1.1149 - acc: 0.6114
Epoch 4/100
50000/50000 [==============================] - 9s 183us/step - loss: 1.0393 - acc: 0.6392
Epoch 5/100
50000/50000 [==============================] - 9s 182us/step - loss: 0.9718 - acc: 0.6619
Epoch 6/100
50000/50000 [==============================] - 10s 202us/step - loss: 0.9176 - acc: 0.6815
Epoch 7/100
50000/50000 [==============================] - 9s 188us/step - loss: 0.8723 - acc: 0.6949
Epoch 8/100
50000/50000 [==============================] - 9s 189us/step - loss: 0.8227 - acc: 0.7131
Epoch 9/100
50000/50000 [==============================] - 10s 192us/step - loss: 0.7750 - acc: 0.7307
Epoch 10/100
50000/50000 [==============================] - 9s 186us/step - los

50000/50000 [==============================] - 9s 184us/step - loss: 0.0448 - acc: 0.9852
Epoch 81/100
50000/50000 [==============================] - 9s 188us/step - loss: 0.0546 - acc: 0.9823
Epoch 82/100
50000/50000 [==============================] - 9s 188us/step - loss: 0.0511 - acc: 0.9831
Epoch 83/100
50000/50000 [==============================] - 9s 188us/step - loss: 0.0510 - acc: 0.9830
Epoch 84/100
50000/50000 [==============================] - 9s 184us/step - loss: 0.0468 - acc: 0.9851
Epoch 85/100
50000/50000 [==============================] - 9s 181us/step - loss: 0.0480 - acc: 0.9846
Epoch 86/100
50000/50000 [==============================] - 9s 181us/step - loss: 0.0440 - acc: 0.9851
Epoch 87/100
50000/50000 [==============================] - 9s 182us/step - loss: 0.0449 - acc: 0.9854
Epoch 88/100
50000/50000 [==============================] - 9s 183us/step - loss: 0.0514 - acc: 0.9837
Epoch 89/100
50000/50000 [==============================] - 9s 183us/step - loss: 0.04

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [12]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[6.4942014e-01, 1.2084073e-09, 1.1689144e-01, 2.3368759e-01,
        3.9883300e-08, 3.3465529e-08, 9.0245744e-10, 2.1450603e-08,
        5.2482659e-07, 1.7796738e-07]], dtype=float32)

In [15]:
mean_train

120.70756512369792

# 參考答案: